In [185]:
# Initial imports
import pandas as pd


In [186]:
# Load Data Set
file_path = "../CSV/Twice_Cleaned_Country.csv"
book_crossing = pd.read_csv(file_path, low_memory=True)
book_crossing

,User_ID,Age,Book_Rating,ISBN,Book_Author,Year_Of_Publication,Publisher,Country
0,161968.0,25.0,1.0,394580567,Robert Fulghum,1989.0,Random House Inc,Afghanistan
1,56235.0,80.0,2.0,60740450,Gabriel Garcia Marquez,2004.0,Perennial,Afghanistan
2,207976.0,65.0,9.0,812925238,Robert S. McNamara,1995.0,Random House Inc,Afghanistan
3,108856.0,60.0,4.0,671019937,John Walsh,1998.0,Atria,United States of America
4,57304.0,46.0,8.0,1570717257,Tony Parsons,2001.0,Sourcebooks Landmark,Canada
...,...,...,...,...,...,...,...,...
258151,259223.0,82.0,8.0,553234811,Jean M. Auel,1983.0,Bantam Doubleday Dell,United States of America
258152,259223.0,82.0,8.0,553227750,Jean Auel,1982.0,Bantam Doubleday Dell,United States of America
258153,259223.0,82.0,7.0,449244741,James Michener,1982.0,Ballantine Books,United States of America
258154,259223.0,82.0,8.0,425126455,Dale Brown,1995.0,Berkley Publishing Group,United States of America


In [187]:
book_crossing.dtypes

User_ID                float64
Age                    float64
Book_Rating            float64
ISBN                    object
Book_Author             object
Year_Of_Publication    float64
Publisher               object
Country                 object
dtype: object

In [188]:
bc_str_id = book_crossing.copy()
bc_str_id.User_ID = bc_str_id.User_ID.astype('str')
bc_str_id.dtypes

User_ID                 object
Age                    float64
Book_Rating            float64
ISBN                    object
Book_Author             object
Year_Of_Publication    float64
Publisher               object
Country                 object
dtype: object

In [189]:
newgroup = bc_str_id.groupby('ISBN')
newgroup.nunique()

,User_ID,Age,Book_Rating,Book_Author,Year_Of_Publication,Publisher,Country
ISBN,,,,,,,
000104799X,2,2,2,1,1,1,1
000160418X,1,1,1,1,1,1,1
000215871X,1,1,1,1,1,1,1
000221766X,1,1,1,1,1,1,1
000222674X,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...
B0001FZGBC,1,1,1,1,1,1,1
B0001FZGPI,1,1,1,1,1,1,1
B0001FZGRQ,1,1,1,1,1,1,1


In [190]:
# Find user count per ISBN
isbn_user_count = bc_str_id.groupby(['ISBN'], as_index = False)['User_ID'].nunique()
isbn_user_count

,ISBN,User_ID
0,000104799X,2
1,000160418X,1
2,000215871X,1
3,000221766X,1
4,000222674X,1
...,...,...
115475,B0001FZGBC,1
115476,B0001FZGPI,1
115477,B0001FZGRQ,1
115478,B0001GMSV2,2


In [191]:
# # Find the median rating of each book
median_book_rating = bc_str_id.groupby('ISBN', as_index = False)['Book_Rating'].median()
median_book_rating

,ISBN,Book_Rating
0,000104799X,7.5
1,000160418X,7.0
2,000215871X,7.0
3,000221766X,8.0
4,000222674X,9.0
...,...,...
115475,B0001FZGBC,7.0
115476,B0001FZGPI,7.0
115477,B0001FZGRQ,9.0
115478,B0001GMSV2,8.0


In [192]:
# Find median Age by ISBN
median_reader_age = bc_str_id.groupby('ISBN', as_index = False)['Age'].median()
median_reader_age

,ISBN,Age
0,000104799X,59.0
1,000160418X,61.0
2,000215871X,42.0
3,000221766X,60.0
4,000222674X,51.0
...,...,...
115475,B0001FZGBC,50.0
115476,B0001FZGPI,26.0
115477,B0001FZGRQ,46.0
115478,B0001GMSV2,28.5


In [193]:
# ISBN country count
isbn_country_count = bc_str_id.groupby(['ISBN'], as_index = False)['Country'].nunique()
isbn_country_count

,ISBN,Country
0,000104799X,1
1,000160418X,1
2,000215871X,1
3,000221766X,1
4,000222674X,1
...,...,...
115475,B0001FZGBC,1
115476,B0001FZGPI,1
115477,B0001FZGRQ,1
115478,B0001GMSV2,2


In [194]:
isbn_info = pd.merge(median_reader_age, isbn_country_count, how='inner', on='ISBN')
isbn_info

,ISBN,Age,Country
0,000104799X,59.0,1
1,000160418X,61.0,1
2,000215871X,42.0,1
3,000221766X,60.0,1
4,000222674X,51.0,1
...,...,...,...
115475,B0001FZGBC,50.0,1
115476,B0001FZGPI,26.0,1
115477,B0001FZGRQ,46.0,1
115478,B0001GMSV2,28.5,2


In [195]:
isbn_info = isbn_info.merge(isbn_user_count, how='inner', on='ISBN')
isbn_info

,ISBN,Age,Country,User_ID
0,000104799X,59.0,1,2
1,000160418X,61.0,1,1
2,000215871X,42.0,1,1
3,000221766X,60.0,1,1
4,000222674X,51.0,1,1
...,...,...,...,...
115475,B0001FZGBC,50.0,1,1
115476,B0001FZGPI,26.0,1,1
115477,B0001FZGRQ,46.0,1,1
115478,B0001GMSV2,28.5,2,2


In [196]:
isbn_info = isbn_info.merge(median_book_rating, how='inner', on='ISBN')
isbn_info

,ISBN,Age,Country,User_ID,Book_Rating
0,000104799X,59.0,1,2,7.5
1,000160418X,61.0,1,1,7.0
2,000215871X,42.0,1,1,7.0
3,000221766X,60.0,1,1,8.0
4,000222674X,51.0,1,1,9.0
...,...,...,...,...,...
115475,B0001FZGBC,50.0,1,1,7.0
115476,B0001FZGPI,26.0,1,1,7.0
115477,B0001FZGRQ,46.0,1,1,9.0
115478,B0001GMSV2,28.5,2,2,8.0


In [197]:
# first_author = bc_str_id.groupby('ISBN', as_index = False)['Book_Author'].first()
# first_author

In [198]:
# isbn_info = isbn_info.merge(first_author, how='inner', on='ISBN')
# isbn_info

In [199]:
# first_pub = bc_str_id.groupby('ISBN', as_index = False)['Publisher'].first()
# first_pub

In [200]:
# isbn_info = isbn_info.merge(first_pub, how='inner', on='ISBN')
# isbn_info

In [201]:
year = bc_str_id.groupby('ISBN', as_index = False)['Year_Of_Publication'].first()
year

,ISBN,Year_Of_Publication
0,000104799X,1994.0
1,000160418X,1984.0
2,000215871X,1992.0
3,000221766X,1987.0
4,000222674X,1982.0
...,...,...
115475,B0001FZGBC,2002.0
115476,B0001FZGPI,2001.0
115477,B0001FZGRQ,2001.0
115478,B0001GMSV2,2002.0


In [202]:
isbn_info = isbn_info.merge(year, how='inner', on='ISBN')
isbn_info

,ISBN,Age,Country,User_ID,Book_Rating,Year_Of_Publication
0,000104799X,59.0,1,2,7.5,1994.0
1,000160418X,61.0,1,1,7.0,1984.0
2,000215871X,42.0,1,1,7.0,1992.0
3,000221766X,60.0,1,1,8.0,1987.0
4,000222674X,51.0,1,1,9.0,1982.0
...,...,...,...,...,...,...
115475,B0001FZGBC,50.0,1,1,7.0,2002.0
115476,B0001FZGPI,26.0,1,1,7.0,2001.0
115477,B0001FZGRQ,46.0,1,1,9.0,2001.0
115478,B0001GMSV2,28.5,2,2,8.0,2002.0


In [203]:
cnames = {
    'Age': 'Median_Reader_Age',
    'Country': 'Country_Count',
    'Book_Rating': 'Median_Book_Rating',
    'User_ID': 'Read_Count',
    }

isbn_info = isbn_info.rename(columns=cnames)
isbn_info

,ISBN,Median_Reader_Age,Country_Count,Read_Count,Median_Book_Rating,Year_Of_Publication
0,000104799X,59.0,1,2,7.5,1994.0
1,000160418X,61.0,1,1,7.0,1984.0
2,000215871X,42.0,1,1,7.0,1992.0
3,000221766X,60.0,1,1,8.0,1987.0
4,000222674X,51.0,1,1,9.0,1982.0
...,...,...,...,...,...,...
115475,B0001FZGBC,50.0,1,1,7.0,2002.0
115476,B0001FZGPI,26.0,1,1,7.0,2001.0
115477,B0001FZGRQ,46.0,1,1,9.0,2001.0
115478,B0001GMSV2,28.5,2,2,8.0,2002.0


In [204]:
popular_threshold = 8
unpopular_threshold = 3

pop_str = 'Popular (>= ' + str(popular_threshold) + ')'
unpop_str = 'Unpopular (<= ' + str(unpopular_threshold) + ')'

isbn_info[pop_str] = isbn_info['Median_Book_Rating'] >= popular_threshold
isbn_info[unpop_str] = isbn_info['Median_Book_Rating'] <= unpopular_threshold
isbn_info

,ISBN,Median_Reader_Age,Country_Count,Read_Count,Median_Book_Rating,Year_Of_Publication,Popular (>= 8),Unpopular (<= 3)
0,000104799X,59.0,1,2,7.5,1994.0,False,False
1,000160418X,61.0,1,1,7.0,1984.0,False,False
2,000215871X,42.0,1,1,7.0,1992.0,False,False
3,000221766X,60.0,1,1,8.0,1987.0,True,False
4,000222674X,51.0,1,1,9.0,1982.0,True,False
...,...,...,...,...,...,...,...,...
115475,B0001FZGBC,50.0,1,1,7.0,2002.0,False,False
115476,B0001FZGPI,26.0,1,1,7.0,2001.0,False,False
115477,B0001FZGRQ,46.0,1,1,9.0,2001.0,True,False
115478,B0001GMSV2,28.5,2,2,8.0,2002.0,True,False


In [205]:
isbn_info.to_csv('../CSV/sml_isbn_info.csv', index=False)